In [10]:
import hashlib

class MerkleTree:
    def __init__(self, transactions):
        self.transactions = transactions
        self.tree = self.build_tree()

    def build_tree(self):
        if len(self.transactions) % 2 != 0:
            self.transactions.append(self.transactions[-1])  # Duplicate last transaction if odd number

        tree = [None] * (2 * len(self.transactions) - 1)

        # Hash leaves
        for i in range(len(self.transactions)):
            tree[i + len(self.transactions) - 1] = hashlib.sha256(self.transactions[i].encode()).hexdigest()

        # Build tree upwards
        for i in range(len(self.transactions) - 2, -1, -1):
            left_child = tree[2 * i + 1]
            right_child = tree[2 * i + 2]
            tree[i] = hashlib.sha256((left_child + right_child).encode()).hexdigest()

        return tree

    def get_root(self):
        return self.tree[0]


In [11]:
import time

class Block:
    def __init__(self, previous_hash, transactions, coinbase, difficulty):
        self.previous_hash = previous_hash
        self.transactions = transactions
        self.coinbase = coinbase  # Reward transaction
        self.merkle_root = self.calculate_merkle_root()
        self.nonce = 0
        self.timestamp = time.time()
        self.difficulty = difficulty
        self.hash = self.mine_block()

    def calculate_merkle_root(self):
        # Include coinbase transaction in the Merkle tree
        all_transactions = [self.coinbase] + self.transactions
        merkle_tree = MerkleTree(all_transactions)
        return merkle_tree.get_root()

    def hash_block(self):
        # Combine block header attributes and return the hash
        block_header = (self.previous_hash + self.merkle_root + str(self.nonce) + str(self.timestamp)).encode()
        return hashlib.sha256(block_header).hexdigest()

    def mine_block(self):
        while True:
            hash_value = self.hash_block()
            if hash_value[:self.difficulty] == '0' * self.difficulty:
                return hash_value
            self.nonce += 1

    def print_block(self):
        print(f"Block Hash: {self.hash}")
        print(f"Previous Hash: {self.previous_hash}")
        print(f"Merkle Root: {self.merkle_root}")
        print(f"Nonce: {self.nonce}")
        print(f"Transactions: {self.transactions}")
        print(f"Coinbase: {self.coinbase}\n")


In [12]:
class Blockchain:
    def __init__(self, difficulty=4):
        self.chain = []
        self.difficulty = difficulty
        self.create_genesis_block()

    def create_genesis_block(self):
        # Create the genesis block with a coinbase transaction
        genesis_block = Block("0" * 64, [], "Genesis Block Reward", self.difficulty)
        self.chain.append(genesis_block)

    def add_block(self, transactions, coinbase):
        previous_hash = self.chain[-1].hash
        new_block = Block(previous_hash, transactions, coinbase, self.difficulty)
        self.chain.append(new_block)

    def validate_transactions(self, transactions):
        # Basic validation: Ensure transactions are non-empty and unique
        for transaction in transactions:
            if len(transaction) == 0:
                return False
            if any(transaction == block.transactions for block in self.chain):
                return False
        return True

    def print_chain(self):
        for block in self.chain:
            block.print_block()


In [13]:
# Crear la blockchain y agregar bloques
blockchain = Blockchain(difficulty=4)

# Añadir bloques con transacciones válidas
if blockchain.validate_transactions(["tx1", "tx2"]):
    blockchain.add_block(["tx1", "tx2"], "Miner's Reward")

if blockchain.validate_transactions(["tx3", "tx4"]):
    blockchain.add_block(["tx3", "tx4"], "Miner's Reward")

# Imprimir la cadena completa
blockchain.print_chain()


Block Hash: 00002a890509564973db947a193d2c71b5c00e34b03130e2b6c80242ab384130
Previous Hash: 0000000000000000000000000000000000000000000000000000000000000000
Merkle Root: 234f4561d66e746ff6bc41afd3824853cbedbd2c58cc3c161aef295cf6865ad6
Nonce: 39060
Transactions: []
Coinbase: Genesis Block Reward

Block Hash: 0000dbc7ffe03a37b74748d482c04f729eebd0cb35019c50ca550b68897bab59
Previous Hash: 00002a890509564973db947a193d2c71b5c00e34b03130e2b6c80242ab384130
Merkle Root: cec4c7dd5220d777fb2a6e3860bd193f7fba881821ac253a0c44b1dd11b59254
Nonce: 25573
Transactions: ['tx1', 'tx2']
Coinbase: Miner's Reward

Block Hash: 0000200c0ee5dbccfe44cbaf454781dca7a0fea3690689190e1f058b0253a674
Previous Hash: 0000dbc7ffe03a37b74748d482c04f729eebd0cb35019c50ca550b68897bab59
Merkle Root: 05970d76c29f3604baa42e41a54e18bc090730aa8e92f07627c68ed967913c17
Nonce: 7892
Transactions: ['tx3', 'tx4']
Coinbase: Miner's Reward

